In [2]:
import os

In [3]:
os.chdir("../")

In [4]:
import os
import dotenv
dotenv.load_dotenv()

True

In [13]:
from pydantic import BaseModel
from pathlib import Path

class IngestConfig(BaseModel):
    connection_string: str
    table_name: str

    class Config:
        frozen = True  # To make the model immutable like the frozen dataclass


In [14]:
from Ingest.constants import *
from Ingest.utils.common import read_yaml, get_microsoft_data, sanitize_column_name, send_dataframe_to_azure_table

In [15]:
class ConfigurationManager:
    def __init__(
        self,
        params_filepath = PARAMS_FILE_PATH):

        self.params = read_yaml(params_filepath)


    
    def get_ingestion_config(self) -> IngestConfig:

        config = self.params


        ingest_config = IngestConfig(

            table_name=config.table_name,

            connection_string=os.getenv('CONNECTION_STRING'),
            
        )

        return ingest_config

In [16]:
class ingest:
    def __init__(self, config: IngestConfig):
        self.config = config
    

    def ingest_data(self):
        microsoft_data = get_microsoft_data(period="1y")
        send_dataframe_to_azure_table(microsoft_data, self.config.connection_string, self.config.table_name)


In [17]:
config = ConfigurationManager()
ingest_config = config.get_ingestion_config()
ingesting = ingest(config=ingest_config)
ingest_data = ingesting.ingest_data()

[2024-08-19 19:02:11,265: INFO: common: yaml file: params.yaml loaded successfully]
[2024-08-19 19:02:11,376: INFO: _universal: Request URL: 'https://csm1klagsto902.table.core.windows.net/Tables'
Request method: 'POST'
Request headers:
    'Content-Type': 'application/json;odata=nometadata'
    'Content-Length': '26'
    'x-ms-version': 'REDACTED'
    'DataServiceVersion': 'REDACTED'
    'Accept': 'application/json;odata=minimalmetadata'
    'x-ms-client-request-id': 'c6922dac-5e4c-11ef-ad92-000d3abb4ec5'
    'x-ms-date': 'REDACTED'
    'Date': 'Mon, 19 Aug 2024 17:02:11 GMT'
    'User-Agent': 'azsdk-python-data-tables/12.5.0 Python/3.9.16 (Windows-10-10.0.17763-SP0)'
    'Authorization': 'REDACTED'
A body is sent with the request]
[2024-08-19 19:02:11,540: INFO: _universal: Response status: 409
Response headers:
    'Cache-Control': 'no-cache'
    'Transfer-Encoding': 'chunked'
    'Content-Type': 'application/json;odata=minimalmetadata;streaming=true;charset=utf-8'
    'Server': 'Win